In [1]:
import pandas as pd
from preProccesPuzzle import PreProcess
from clue_classifier import ClueClassifier
from constraints import Constraint, IdentityConstrain, NextToConstrain, DistanceConstrain,RightConstrain, LeftConstrain, DirectRightConstrain, DirectLeftConstrain, PositionAbsoluteConstrain, PositionAbsoluteNegativeConstrain
from constraint_solver import ConstraintSolver

In [2]:
gridmode = pd.read_parquet("Gridmode-00000-of-00001.parquet")
mc = pd.read_parquet("mc-00000-of-00001.parquet")
ppp = PreProcess()
classifier = ClueClassifier()

In [ ]:
puzzle = mc.puzzle.iloc[2].lower()

attrs, clues = ppp.proccess(puzzle)

print(puzzle)

constrains: list[Constraint] = []

for c in clues:
    clue, clue_type = classifier.classify(c)

    if clue_type == "IDENTITY":
        constrains.append(IdentityConstrain(attrs, clue))
    if clue_type == "NEXT_TO":
        constrains.append(NextToConstrain(attrs, clue))
    if clue_type == "LEFT":
        constrains.append(LeftConstrain(attrs, clue))
    if clue_type == "RIGHT":
        constrains.append(RightConstrain(attrs, clue))
    if clue_type == "DISTANCE":
        constrains.append(DistanceConstrain(attrs, clue))
    if clue_type == "DIRECT_LEFT":
        constrains.append(DirectLeftConstrain(attrs, clue))
    if clue_type == "DIRECT_RIGHT":
        constrains.append(DirectRightConstrain(attrs, clue))
    if clue_type == "POSITION_ABSOLUTE":
        constrains.append(PositionAbsoluteConstrain(attrs, clue))
    if clue_type == "POSITION_ABSOLUTE_NEGATIVE":
        constrains.append(PositionAbsoluteNegativeConstrain(attrs, clue))
    if clue_type == "UNKNOWN":
        raise TypeError

Cs = ConstraintSolver(attrs, constrains)

solution = Cs.solve()

Cs.print_solution(solution)



there are 3 houses, numbered 1 to 3 from left to right, as seen from across the street. each house is occupied by a different person. each house has a unique attribute for each of the following characteristics:
 - each person has a unique name: `eric`, `peter`, `arnold`
 - each person has a unique level of education: `associate`, `high school`, `bachelor`
 - each person prefers a unique type of vacation: `city`, `beach`, `mountain`
 - people have unique favorite music genres: `classical`, `rock`, `pop`
 - each person has an occupation: `engineer`, `doctor`, `teacher`
 - people have unique favorite sports: `tennis`, `basketball`, `soccer`

## clues:
1. peter is in the first house.
2. the person with a high school diploma is directly left of the person who loves beach vacations.
3. the person who is a doctor is not in the third house.
4. the person who loves rock music and the person who loves pop music are next to each other.
5. the person who is a teacher is the person who enjoys mount